In [1]:
import numpy as np
from scipy import *

In [5]:
#ke por interpolação linear

ka = 10 #W/m°C 
kb = 1 #W/m°C
La = 73 #mm
Lb = 17 #mm
h = 80 ##W/m²°C
T_infinito = 20 #°C
q0 = 4000 #W/m
A = 1 #m²; área transversal hipotética
q_ponto = 0 #problema sem geração

'''
for k in [ka,kb]:
    for 

'''

meshA = 9
meshB = 5

Matrix_coef_a = np.zeros((meshA+meshB,meshA+meshB))
Matrix_coef_b = np.zeros((meshA+meshB,1))

deltax_A = La/(meshA-0.5)
deltax_B = Lb/(meshB-0.5)

for volume in range(meshA):
    if (volume == 0):
        ae = ka*A/(deltax_A/2)
        ap = ae
        b = q_ponto*A*deltax_A/2 + q0*A
        
        Matrix_coef_a[1,1] = ap
        Matrix_coef_a[1,2] = -ae
        Matrix_coef_b[1,1] = b
        
    if (volume != 0) and (volume != meshA-1):
        ae = ka*A/(deltax_A)
        aw = ae
        ap = ae + aw
        b = q_ponto*A*deltax_A
        
        Matrix_coef_a[volume,volume] = ap
        Matrix_coef_a[volume,volume+1] = -ae
        Matrix_coef_a[volume,volume-1] = -aw
        Matrix_coef_b[volume,1] = b
        
    if (volume == meshA):
        fe = deltax_B/(deltax_A+deltax_B)  #verificar
        ke = fe*ka + (1-fe)*kb
        
        ae = ke*A/(deltax_A)
        aw = ka*A/(deltax_A)
        ap = ae + aw
        b = q_ponto*A*deltax_A
        
        
        
